# Análisis de Pose en Actividades Deportivas
Se deben instalar las librerías antes de pasar a usar el modelo.

# Importación de Datos e Instalación de Librerías

In [7]:
# Instalacion de ultralytics
pip install ultralytics


In [2]:
# Instalacion de timm
pip install timm torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.4 MB/s eta 0:00:00


In [3]:
# Revisar que ultralytics esté funcionando bien
import cv2
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.2/78.2 GB disk)


In [30]:
# Importación de datos de Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Análisis y Organización del Dataset


In [42]:
# Función para convertir videos en frames con poses aisladas en fondo negro
from PIL import Image
from ultralytics import YOLO
import numpy as np

 # La función recibe 'name' como nombre para el archivo, 'path' como ubicación del archivo y 'end' como la ubicación donde se quiere enviar.
def crear_dataset(name, path, end):
  model = YOLO('yolov8n-pose.pt') # Modelo YOLO Pose para la detección
  results = model(path) # Se aplica detección en el video
  n=0
  j=0
  while j<len(results):
    n = n+1
    original = results[j].orig_shape
    h = int(original[0])
    w = int(original[1])
    black_img = np.zeros((h,w,3),dtype=np.uint8) # Se genera imagen negra con alto y largo de la imagen original
    img_array = results[j].plot(img=black_img, labels=False, boxes=False, probs=False)
    im = Image.fromarray(img_array[..., ::-1])
    # im.show() en caso de querer ver las imágenes
    im_path = end +'/results_'+str(name)+'_'+str(n)+'.jpg'
    im.save(im_path)
    j = j+1 # Frame a frame (se puede hacer saltando frames si son muchos videos)

In [41]:
# Función para copiar cierto número de imágenes de una carpeta a otra
import os
import shutil
def copiar_dataset(path,end):
  files = os.listdir(path)
  for file in files[:200]:
    start = os.path.join(path, file)
    destiny = os.path.join(end, file)
    shutil.copy(start, destiny)

In [40]:
# Separar frames de videos en imágenes
import cv2
import os

def vid_to_frame(video_path, output_path):
    # Abre el video
    cap = cv2.VideoCapture(video_path)
    # Guardar cada frame del video como imagen
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_path, f"frame_{frame_count:04d}.png")
        cv2.imwrite(frame_path, frame)
        frame_count += 1
    # Cierra el video
    cap.release()
    print(f"Se extrajeron {frame_count} frames y se guardaron en {output_path}.")

#Entrenamiento

In [31]:
import torch
import timm
import torch.optim as optim

# Se registra la cantidad de ejercicios distintos, en este caso 5
num_classes = 4

# Se parte de un modelo preentrenado, en este caso una mobilenet
model = timm.create_model('timm/resnet50.a1_in1k', pretrained=True, num_classes=num_classes)

# Herramientas para la clasificación
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
import os
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

cuda = torch.cuda.is_available()
#Importación datasets
train_data_path = '/content/drive/MyDrive/Base de Datos Duckietown/Dataset3/Train'
val_data_path = '/content/drive/MyDrive/Base de Datos Duckietown/Dataset3/Test'

#Transformaciones a las imágenes (revisar Resize!)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

#ImageFolders para los datasets (pendiente agregar transforms)
train_dataset = ImageFolder(root=train_data_path, transform=transform)
val_dataset = ImageFolder(root=val_data_path, transform=transform)

print(train_dataset.classes)
print(val_dataset.classes)

#Definir dataset_loaders
batch_size = 64
kwargs = {'num_workers': 2, 'pin_memory': True} if cuda else {}
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, **kwargs)

#Número de épocas
num_epochs = 10

if cuda:
    model.cuda()
#Entrenamiento del modelo con set Train
for epoch in range(num_epochs):
    total_loss = 0
    val_total_loss = 0
    for inputs, labels in train_loader:
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()

#Validación del modelo con set Test
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs = val_inputs.cuda()
            val_labels = val_labels.cuda()
            val_outputs = model(val_inputs)
            val_loss = criterion(val_outputs, val_labels)
            val_total_loss += val_loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}, Val Loss: {val_total_loss/len(val_loader)}')

#Guardado del modelo
model_path = '/content/drive/MyDrive/Base de Datos Duckietown/Modelo4/duckietown-pose-model.pth'
torch.save(model.state_dict(), model_path)


['PesoMuerto', 'Press', 'Squat', 'Standing']
['PesoMuerto', 'Press', 'Squat', 'Standing']
Epoch 1/10, Loss: 0.5153058572775788, Val Loss: 2.010684698820114
Epoch 2/10, Loss: 0.0060094143522696365, Val Loss: 2.496789370264326
Epoch 3/10, Loss: 0.000926949630310345, Val Loss: 2.61347034573555
Epoch 4/10, Loss: 0.0002528099047830673, Val Loss: 2.6801862801824297
Epoch 5/10, Loss: 0.0002291427189009954, Val Loss: 2.734697405781065
Epoch 6/10, Loss: 0.0001538270619305506, Val Loss: 2.7721207099301473
Epoch 7/10, Loss: 0.00013384347254975215, Val Loss: 2.795907139778137
Epoch 8/10, Loss: 0.00013520376254476746, Val Loss: 2.8266061586993083
Epoch 9/10, Loss: 9.224263622955832e-05, Val Loss: 2.8551370927265713
Epoch 10/10, Loss: 7.144262990213638e-05, Val Loss: 2.884343441043581


# Uso del Modelo para la detección de poses

In [34]:
# Se definen las herramientas a usar para el programa final
from ultralytics import YOLO
import numpy as np
import torch
import timm
from torchvision import transforms
from PIL import Image, ImageDraw, ImageFont
import cv2

# Carga del modelo
model_pose = YOLO('yolov8n-pose.pt')
model_path = '/content/drive/MyDrive/Base de Datos Duckietown/Modelo4/duckietown-pose-model.pth'
num_classes = 4
model = timm.create_model('timm/resnet50.a1_in1k', pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load(model_path)) # Se agregan los pesos del entrenamiento
model.eval() # Se deja el modelo en modo evaluación

# Función para aislar en un fondo negro la pose de una imagen.
def isolate_pose(path):
  model = YOLO('yolov8n-pose.pt') # Modelo YOLO Pose para la detección
  results = model(path)
  original = results[0].orig_shape
  h = int(original[0])
  w = int(original[1])
  black_img = np.zeros((h,w,3),dtype=np.uint8) # Se genera imagen negra con alto y largo de la imagen original
  img_array = results[0].plot(img=black_img, labels=False, boxes=False, probs=False)
  im = Image.fromarray(img_array[..., ::-1])
  return im

# Transformaciones a realizar en las imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Preprocesamiento imagen
def preprocess_image(image):
    #image = Image.open(image)
    image = transform(image)
    image = image.unsqueeze(0)
    return image

# Función para predecir con el modelo
def model_predict(model, image):
    with torch.no_grad():
        outputs = model(image)
    return outputs

A continuación se tiene la función con la que se realiza el análisis de pose, sólo se debe correr el código de arriba para que pueda funcionar y usar la función dándole como única variable la ruta hacia la imagen/video que se quiere estudiar.

In [38]:
# Función para detectar el ejercicio que se está realizando a partir de una imagen o video
# La base de datos del entrenamiento actual contiene: persona de pie, sentadilla, flexión, peso muerto y press militar.

def pose_detect(image_path):
  classes = {
    0: 'PesoMuerto',
    1: 'Press',
    2: 'Squat',
    3: 'Standing'}
  pose_img = isolate_pose(image_path)
  pre_img = preprocess_image(pose_img)
  results = model_predict(model, pre_img)
  probabilidades, predicciones = torch.max(results, 1)
  predicted_class = classes[predicciones.item()]
  text = f"Clase predicha: {predicted_class}, Probabilidad: {probabilidades.item()}"
  print(text)
  draw = ImageDraw.Draw(pose_img) # Se escriben los resultados sobre la imagen con la pose
  font = ImageFont.truetype('/content/drive/MyDrive/Base de Datos Duckietown/Roboto-Medium.ttf', size=20) # Se elige la fuente para escribir
  position = (10, 10) # Se define la posición en la que estará el texto
  draw.text(position, text, fill=(255, 255, 255), font=font)
  return pose_img

Y por último, se tiene una función para la detección en tiempo real, esto se debe correr localmente en un dispositivo con GPU disponible y cámara funcional.

In [39]:
# Detección en tiempo real con la función
import cv2

def pose_detect_camera():
    classes = {
        0: 'PesoMuerto',
        1: 'Press',
        2: 'Squat',
        3: 'Standing'
    }

    # Iniciar la captura de video con la cámara
    cap = cv2.VideoCapture(0) # 0 es para la cámara default

    while True:
        # Capturar frame de la cámara
        ret, frame = cap.read()

        # Si no se puede capturar el frame, salir del bucle
        if not ret:
            break

        # Convertir el frame de BGR a RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Convertir el frame a una imagen de PIL
        pil_img = Image.fromarray(rgb_frame)

        # Aplicar el análisis de pose al frame
        pose_img = pose_detect(pil_img)

        # Mostrar el frame resultante con las detecciones
        cv2.imshow('Pose Detection', np.array(pose_img)[:, :, ::-1])

        # Salir del bucle si se presiona la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

# Usar la función para iniciar la detección en tiempo real
pose_detect_camera()